##Generación de caracteristicas para cada péptido


In [ ]:
!pip install propy3
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.3/290.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from Bio import SeqIO
from propy import PyPro
from propy.GetProteinFromUniprot import GetProteinSequence
from google.colab import drive
from sklearn.model_selection import train_test_split

from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

##Rutas de archivos principales

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path1 = '/content/drive/MyDrive/Colab Notebooks/Laboratorio Integrado I/output_fasta_peptides_completo.fasta'
path2 = '/content/drive/MyDrive/Colab Notebooks/Laboratorio Integrado I/df_final.csv'

In [ ]:
df_final = pd.read_csv(path2)
df_sequences = df_final[['id','sequence']]

In [ ]:
df_sequences.head(10)

,id,sequence
0,peptide_0,GIGAVLKVLTTGLPALISWIKRKRQQ
1,peptide_1,GIGKFLHSAKKFGKAFVGEIMNS
2,peptide_2,ILPWKWPWWPWRR
3,peptide_3,KWCFRVCYRGICYRRCR
4,peptide_4,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES
5,peptide_5,FFHHIFRGIVHVGKTIHRLVTG
6,peptide_6,FLPLIGRVLSGIL
7,peptide_7,FWGALAKGALKLIPSLFSSFSKKD
8,peptide_8,GWGSFFKKAAHVGKHVGKAALTHYL
9,peptide_9,ACNFQSCWATCQAQHSIYFRRAFCDRSQCKCVFVRG


In [ ]:
df_sequences.shape

(40840, 2)

In [ ]:
df_sequences.to_csv('df_sequences.csv', index=False)

Ayuda para entender mejor la librería Propy3

In [ ]:
# Crear un objeto de descriptores para probar
des_object = PyPro.GetProDes('ACDEFGHIKLMNPQRSTVWY')

# Obtener ayuda sobre el objeto
help(des_object)

Help on GetProDes in module propy.PyPro object:

class GetProDes(builtins.object)
 |  GetProDes(ProteinSequence: str = '') -> None
 |  
 |  Collect all descriptor calcualtion modules.
 |  
 |  Methods defined here:
 |  
 |  GetAAComp(self) -> Dict[str, float]
 |      Amino acid compositon descriptors (20).
 |      
 |      Examples
 |      --------
 |      >>> from propy.GetProteinFromUniprot import GetProteinSequence
 |      >>> protein = GetProteinSequence(ProteinID="Q9NQ39")
 |      >>> result = GetProDes(protein).GetAAComp()
 |  
 |  GetAAindex1(self, name: str, path: Optional[str] = '.') -> Dict[str, float]
 |      Get the amino acid property values from aaindex1.
 |      
 |      Parameters
 |      ----------
 |      name : str
 |          is the name of amino acid property (e.g., KRIW790103)
 |      
 |      Returns
 |      -------
 |      result is a dict form containing the properties of 20 amino acids
 |      
 |      Examples
 |      --------
 |      >>> from propy.GetProtei

##Calculo de los descriptores PAAC de forma individual

In [ ]:
def read_fasta(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, 'fasta'):
        sequences.append((record.id, str(record.seq)))
    return sequences

def compute_descriptors_test(sequences):
    results = []
    for seq_id, seq in sequences:
        des_object = PyPro.GetProDes(seq)

        # PAAC
        paac = des_object.GetPAAC(lamda=5, weight=0.05)

        # Preparar el resultado como un diccionario
        result = {'id': seq_id}

        result.update(paac)

        results.append(result)

    return pd.DataFrame(results)


In [ ]:
sequences = read_fasta(path1)

df_descriptors = compute_descriptors_test(sequences)

In [ ]:
df_descriptors_paac = df_descriptors

Se exporta el archivo con los descriptores PAAC de los péptidos para no volver a ejecutar la celda por temas de eficiencia.

In [ ]:
df_descriptors_paac.to_csv('df_descriptors_paac.csv', index=False)

##Calculo de las otras carácteristicas fisicoquimicas utilizando Propy3

In [ ]:
def read_fasta(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, 'fasta'):
        sequences.append((record.id, str(record.seq)))
    return sequences

def compute_descriptors(sequences):
    results = []
    for seq_id, seq in sequences:
        des_object = PyPro.GetProDes(seq)

        # Composición de Aminoácidos
        aac = des_object.GetAAComp()

        # Dipeptide Composition (DPC)
        dpc = des_object.GetDPComp()

        # Composition, Transition, Distribution (CTD)
        ctd = des_object.GetCTD()

        #Mbauto
        mbauto = des_object.GetMoreauBrotoAuto()

        # Moranauto
        moranauto = des_object.GetMoranAuto()

        #Geryauto
        geryauto = des_object.GetGearyAuto()

        # Sequence Order Cumulative Number (SOCN)
        socn = des_object.GetSOCN()

        # Quasi-Symmetric Order (QSO)
        qso = des_object.GetQSO()

        # Preparar el resultado como un diccionario
        result = {'id': seq_id}
        result.update(aac)
        result.update(dpc)
        result.update(ctd)
        result.update(mbauto)
        result.update(moranauto)
        result.update(geryauto)
        result.update(socn)
        result.update(qso)
        results.append(result)

    return pd.DataFrame(results)


In [ ]:
sequences = read_fasta(path1)

df_descriptors = compute_descriptors(sequences)

In [ ]:
df_descriptors.iloc[:, 1:]

,A,R,N,D,C,E,Q,G,H,I,...,QSOgrant41,QSOgrant42,QSOgrant43,QSOgrant44,QSOgrant45,QSOgrant46,QSOgrant47,QSOgrant48,QSOgrant49,QSOgrant50
0,7.692,7.692,0.000,0.000,0.000,0.000,7.692,11.538,0.000,11.538,...,0.020989,0.014686,0.012446,0.006902,0.002686,0.000000,0.000000,0.000000,0.000000,0.000000
1,8.696,0.000,4.348,0.000,0.000,4.348,0.000,17.391,4.348,8.696,...,0.009988,0.001179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000,15.385,0.000,0.000,0.000,0.000,0.000,0.000,0.000,7.692,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000,29.412,0.000,0.000,23.529,0.000,0.000,5.882,0.000,5.882,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000,13.514,2.703,5.405,0.000,8.108,2.703,5.405,0.000,8.108,...,0.025547,0.017740,0.022454,0.029931,0.022450,0.007209,0.014647,0.017532,0.013910,0.008937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40835,11.429,5.714,2.857,0.000,0.000,0.000,11.429,5.714,2.857,2.857,...,0.021349,0.032427,0.022858,0.021531,0.023839,0.018923,0.014843,0.016204,0.011915,0.009568
40836,0.000,0.000,15.385,7.692,0.000,7.692,7.692,0.000,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
40837,17.647,17.647,5.882,5.882,0.000,5.882,5.882,0.000,0.000,5.882,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
40838,6.452,3.226,12.903,3.226,0.000,0.000,0.000,3.226,3.226,16.129,...,0.025101,0.027006,0.029928,0.019077,0.018889,0.011021,0.008275,0.012708,0.008093,0.001881


In [ ]:
df_descriptors

,id,A,R,N,D,C,E,Q,G,H,...,QSOgrant41,QSOgrant42,QSOgrant43,QSOgrant44,QSOgrant45,QSOgrant46,QSOgrant47,QSOgrant48,QSOgrant49,QSOgrant50
0,peptide_0,7.692,7.692,0.000,0.000,0.000,0.000,7.692,11.538,0.000,...,0.020989,0.014686,0.012446,0.006902,0.002686,0.000000,0.000000,0.000000,0.000000,0.000000
1,peptide_1,8.696,0.000,4.348,0.000,0.000,4.348,0.000,17.391,4.348,...,0.009988,0.001179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,peptide_2,0.000,15.385,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,peptide_3,0.000,29.412,0.000,0.000,23.529,0.000,0.000,5.882,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,peptide_4,0.000,13.514,2.703,5.405,0.000,8.108,2.703,5.405,0.000,...,0.025547,0.017740,0.022454,0.029931,0.022450,0.007209,0.014647,0.017532,0.013910,0.008937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40835,peptide_43335,11.429,5.714,2.857,0.000,0.000,0.000,11.429,5.714,2.857,...,0.021349,0.032427,0.022858,0.021531,0.023839,0.018923,0.014843,0.016204,0.011915,0.009568
40836,peptide_43336,0.000,0.000,15.385,7.692,0.000,7.692,7.692,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
40837,peptide_43337,17.647,17.647,5.882,5.882,0.000,5.882,5.882,0.000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
40838,peptide_43338,6.452,3.226,12.903,3.226,0.000,0.000,0.000,3.226,3.226,...,0.025101,0.027006,0.029928,0.019077,0.018889,0.011021,0.008275,0.012708,0.008093,0.001881


In [ ]:
df_descriptors.shape

(40840, 1478)

##Exportamos el csv descriptors para no volver a ejecutar la función de calcular los descriptores cada vez que se ejecute el notebook.

In [ ]:
df_descriptors.to_csv('df_descriptors.csv', index=False)

##Descriptores fisicoquimicos con R


Para ejecutar las siguiente celdas, cambiar el entorno de ejecución de python3 a R

In [ ]:
# Instala un paquete en R
install.packages("Peptides")
library(Peptides)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
df <- read.csv('df_sequences.csv')

In [ ]:
head(df, 10)

,id,sequence
,<chr>,<chr>
1,peptide_0,GIGAVLKVLTTGLPALISWIKRKRQQ
2,peptide_1,GIGKFLHSAKKFGKAFVGEIMNS
3,peptide_2,ILPWKWPWWPWRR
4,peptide_3,KWCFRVCYRGICYRRCR
5,peptide_4,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES
6,peptide_5,FFHHIFRGIVHVGKTIHRLVTG
7,peptide_6,FLPLIGRVLSGIL
8,peptide_7,FWGALAKGALKLIPSLFSSFSKKD
9,peptide_8,GWGSFFKKAAHVGKHVGKAALTHYL


In [ ]:
#cargar csv
df <- read.csv("df_sequences.csv")

# Definir la función para calcular propiedades peptídicas
calcular_propiedades_peptidicas <- function(df, secuencia_col) {
  # Aplicar las funciones del paquete Peptides a la columna de secuencias
  df$Longitud <- sapply(df[[secuencia_col]], nchar)
  df$Indice_Boman <- sapply(df[[secuencia_col]], boman)
  df$Carga <- sapply(df[[secuencia_col]], charge)  # Carga
  df$Punto_Isoelectrico <- sapply(df[[secuencia_col]], pI)  # Punto isoeléctrico
  df$Hidrofobicidad <- sapply(df[[secuencia_col]], hydrophobicity)  # Hidrofobicidad

  return(df)
}

df_properties <- calcular_propiedades_peptidicas(df, "sequence")

In [ ]:
tail(df_properties, 10)

,id,sequence,Longitud,Indice_Boman,Carga,Punto_Isoelectrico,Hidrofobicidad
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
40831,peptide_43330,EKLGANAILGVSLAVAKAAANELRMPLFRYLG,32,0.2512500,2.0000874404,10.239781,0.5406250
40832,peptide_43331,ANKEALGIYVADLDGNEYGVGDYEYPFTI,29,0.9868966,-4.9990475706,3.532167,-0.2689655
40833,peptide_43332,IFKLALACNKV,11,-0.8127273,1.9354195158,9.676251,1.2636364
40834,peptide_43333,FKHGIPIDADYVFDVRFLPNPHWDPKLRPMT,31,1.6245161,0.1801413133,7.704007,-0.4806452
40835,peptide_43334,FTSMPVYAIIVANFCRSWTFYLLLISQPAYF,31,-0.5822581,0.9334550295,8.487526,1.0000000
40836,peptide_43335,SNQKQALAQPQTFAHIRVFFRPLPFAFFGTVGFYS,35,0.8031429,3.0877413196,11.530512,0.1085714
40837,peptide_43336,ETVDLKLNLQSNK,13,2.5092308,-0.0003841614,6.492972,-0.8615385
40838,peptide_43337,DRALAPNIRSAKQMVER,17,3.5152941,1.9999009385,11.378050,-0.8235294
40839,peptide_43338,HDNKSKNNTFNGAIAYVIPFLLFIFVIRSLI,31,0.3725806,2.0878960812,10.243799,0.5967742


In [ ]:
write.csv(df_properties,"df_properties.csv", row.names = FALSE)

##Características completas

Importamos los dfs y unimos para tener el df completo

In [ ]:
df_properties = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Laboratorio Integrado I/df_properties_final.csv')
df_descriptors_paac = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Laboratorio Integrado I/df_descriptors_paac.csv')

In [ ]:
df_all_properties = df_properties.merge(df_descriptors_paac, on='id', how='right')

In [ ]:
df_all_properties.columns

Index(['id', 'sequence', 'Longitud', 'Indice_Boman', 'Carga',
       'Punto_Isoelectrico', 'Hidrofobicidad', 'A', 'R', 'N',
       ...
       'PAAC16', 'PAAC17', 'PAAC18', 'PAAC19', 'PAAC20', 'PAAC21', 'PAAC22',
       'PAAC23', 'PAAC24', 'PAAC25'],
      dtype='object', length=1509)

In [ ]:
df_all_properties.head()

,id,sequence,Longitud,Indice_Boman,Carga,Punto_Isoelectrico,Hidrofobicidad,A,R,N,...,PAAC16,PAAC17,PAAC18,PAAC19,PAAC20,PAAC21,PAAC22,PAAC23,PAAC24,PAAC25
0,peptide_0,GIGAVLKVLTTGLPALISWIKRKRQQ,26,0.570000,4.997093,12.546070,0.273077,7.692,7.692,0.000,...,2.646,5.293,2.646,0.000,5.293,5.054,5.918,4.968,7.060,8.189
1,peptide_1,GIGKFLHSAKKFGKAFVGEIMNS,23,0.419130,3.089488,10.802733,0.082609,8.696,0.000,4.348,...,5.567,0.000,0.000,0.000,2.784,7.414,9.107,6.674,6.124,6.661
2,peptide_2,ILPWKWPWWPWRR,13,1.069231,2.997683,12.516461,-1.069231,0.000,15.385,0.000,...,0.000,0.000,20.956,0.000,0.000,9.821,7.018,9.467,11.736,7.473
3,peptide_3,KWCFRVCYRGICYRRCR,17,3.535294,5.748073,10.068130,-0.517647,0.000,29.412,0.000,...,0.000,0.000,3.161,6.322,3.161,10.336,9.635,10.674,7.020,8.600
4,peptide_4,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,37,3.000000,6.002416,11.346921,-0.724324,0.000,13.514,2.703,...,3.164,1.582,0.000,0.000,3.164,8.037,10.856,7.481,5.494,9.594


In [ ]:
resultados = {}

df = df_all_properties.iloc[:,2:]

for columna in df.columns:
    stat, p = shapiro(df[columna])
    resultados[columna] = p
    print(f'Columna: {columna} - Estadístico: {stat}, p-value: {p}')

    if p > 0.05:
        print(f' -> {columna}: Los datos parecen tener una distribución normal.')
    else:
        print(f' -> {columna}: Los datos no siguen una distribución normal.')

#Crear un DataFrame con los resultados para visualizar mejor
df_resultados = pd.DataFrame(resultados.items(), columns=['Columna', 'p-value'])
print(df_resultados)


/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 40840.
  res = hypotest_fun_out(*samples, **kwds)


Columna: Longitud - Estadístico: 0.7866853184325882, p-value: 5.986887475725702e-112
 -> Longitud: Los datos no siguen una distribución normal.
Columna: Indice_Boman - Estadístico: 0.9782719349929323, p-value: 6.614799239422252e-59
 -> Indice_Boman: Los datos no siguen una distribución normal.
Columna: Carga - Estadístico: 0.9612748889562254, p-value: 1.0749362833465493e-70
 -> Carga: Los datos no siguen una distribución normal.
Columna: Punto_Isoelectrico - Estadístico: 0.9392133434978874, p-value: 1.165036454617871e-80
 -> Punto_Isoelectrico: Los datos no siguen una distribución normal.
Columna: Hidrofobicidad - Estadístico: 0.9937537182191609, p-value: 3.1314983763610813e-37
 -> Hidrofobicidad: Los datos no siguen una distribución normal.
Columna: A - Estadístico: 0.8616660973598931, p-value: 1.4483446826199758e-100
 -> A: Los datos no siguen una distribución normal.
Columna: R - Estadístico: 0.7777775737855118, p-value: 4.6752855073542754e-113
 -> R: Los datos no siguen una distrib

##Varianza de las columnas sin normalizar

In [ ]:
varianza_columnas = df_all_properties.iloc[:, 2:].var()

In [ ]:
varianza_columnas = varianza_columnas.to_numpy()

In [ ]:
varianza_columnas.mean()

967177654.8702816

In [ ]:
for i in varianza_columnas:
  print(i)

316.431101355597
2.670848596382635
12.092109055759616
8.158346235798163
0.7622781325097882
53.922611742430135
79.31591568582512
18.958396407090017
19.721609096102277
36.34374360434339
29.189113907385675
18.379163800937487
46.238127311290135
20.31705709807034
36.2884382980042
75.14652882849741
87.65718119960682
13.408332796215115
32.14309417713571
39.66238353798233
32.96302747266553
25.307166136233153
32.309872225415944
15.77830064262206
36.96380903653257
4.861035284499189
2.366765784319374
0.992637889921613
1.162066730218197
0.9772187389918358
1.5010625612863204
0.9379016308440985
3.061203213172196
1.0569230076131186
2.5156526193761897
5.961575639233495
5.22673716172053
0.6534457119423006
1.8682226689605523
1.4492660086604565
2.304398350892374
1.7439253676596636
0.9164152792416531
0.8197922365857095
2.6649481221003573
2.0451945159290497
8.478455881406372
1.2711333754995564
1.0571131937165636
1.9263672956017857
1.1067986913858667
1.0073871842486315
2.4354370054532417
1.0614457748267598


##Normalización de los datos con MinMax

In [ ]:
scaler = MinMaxScaler()
array_scaled = scaler.fit_transform(df_all_properties.iloc[:,2:])

In [ ]:
df_min_max = pd.DataFrame(array_scaled, columns=df_all_properties.iloc[:,2:].columns)

In [ ]:
df_min_max

,Longitud,Indice_Boman,Carga,Punto_Isoelectrico,Hidrofobicidad,A,R,N,D,C,...,PAAC16,PAAC17,PAAC18,PAAC19,PAAC20,PAAC21,PAAC22,PAAC23,PAAC24,PAAC25
0,0.204301,0.283376,0.403184,0.877668,0.583464,0.106833,0.097162,0.000000,0.000000,0.000000,...,0.037587,0.125689,0.062376,0.000000,0.107063,0.252797,0.283954,0.276215,0.379835,0.403598
1,0.172043,0.274006,0.372405,0.723791,0.559056,0.120778,0.000000,0.083854,0.000000,0.000000,...,0.079080,0.000000,0.000000,0.000000,0.056313,0.387531,0.444310,0.371066,0.329478,0.328290
2,0.064516,0.314383,0.370923,0.875055,0.411451,0.000000,0.194336,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.494012,0.000000,0.000000,0.524949,0.339267,0.526354,0.631409,0.368310
3,0.107527,0.467550,0.415301,0.658950,0.482135,0.000000,0.371518,0.000000,0.000000,0.431369,...,0.000000,0.000000,0.074517,0.194805,0.063939,0.554350,0.470860,0.593462,0.377683,0.423854
4,0.322581,0.434303,0.419405,0.771824,0.455650,0.000000,0.170702,0.052129,0.075670,0.000000,...,0.044945,0.037566,0.000000,0.000000,0.063999,0.423099,0.532257,0.415935,0.295583,0.472844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40835,0.301075,0.297857,0.372376,0.788029,0.562383,0.158736,0.072177,0.055099,0.000000,0.000000,...,0.054747,0.091518,0.000000,0.059378,0.077956,0.352820,0.307940,0.374291,0.310486,0.337112
40836,0.064516,0.403822,0.322549,0.343385,0.438066,0.000000,0.000000,0.296710,0.107687,0.000000,...,0.078341,0.130960,0.000000,0.000000,0.111554,0.394611,0.140645,0.390637,0.279281,0.270281
40837,0.107527,0.466308,0.354824,0.774572,0.442937,0.245097,0.222909,0.113438,0.082348,0.000000,...,0.052090,0.000000,0.000000,0.000000,0.074174,0.383193,0.445014,0.415101,0.370313,0.337555
40838,0.258065,0.271115,0.356244,0.674456,0.624945,0.089611,0.040749,0.248843,0.045164,0.000000,...,0.068625,0.057347,0.000000,0.074415,0.097718,0.260276,0.226228,0.312632,0.242320,0.248546


#Varianza de las columnas después de normalizar los datos

In [ ]:
df_min_max_array = df_min_max.var().to_numpy()

In [ ]:
for i in df_min_max_array:
  print(i)

0.0365858597936854
0.010303162350576727
0.003148106747732544
0.06356094275976452
0.012517895397499512
0.010401738376240968
0.012655286070229558
0.007051322451960852
0.0038653889980289
0.012215739639610933
0.012761237525951633
0.003417644172902604
0.007224707392386193
0.00530162914730077
0.009582559475918928
0.015336026291534447
0.016572810116407734
0.0083802079976345
0.011300447801801162
0.008613695186096325
0.006129544302630499
0.010122866454494141
0.0072696485540407
0.008590465397322854
0.009760902236693894
0.003456736202305812
0.003477019481195809
0.00357206743386372
0.0018593067683479855
0.002956677991482194
0.004541622526981726
0.0033750956977570924
0.004303741016019991
0.0019841373979378124
0.0033828317207920465
0.004509343400761323
0.004196887660383331
0.002351463883378772
0.0017951465608219697
0.002720673943730118
0.0028231702898116155
0.0022597656995139944
0.0014662644467869253
0.0029500719049430795
0.0034532202289030223
0.002750200417329107
0.005537287446783633
0.003845947611

In [ ]:
df_min_max_array.mean()

0.009332034954164712

##Aplicación del primer método de selección

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Aplicar VarianceThreshold
selector = VarianceThreshold(threshold=0.0025)  # Umbral de varianza
X_reduced = selector.fit_transform(df_min_max)

# Crear DataFrame con características originales seleccionadas
X_reduced_df = pd.DataFrame(X_reduced, columns=df_min_max.columns[selector.get_support()])

# Identificar las columnas eliminadas
eliminated_columns = df_min_max.columns[~selector.get_support()]

print("Columnas eliminadas por umbral de varianza:")
print(eliminated_columns)

Columnas eliminadas por umbral de varianza:
Index(['AD', 'AH', 'AM', 'AF', 'AT', 'AW', 'RH', 'RM', 'RF', 'RP',
       ...
       'QSOgrant12', 'QSOgrant13', 'QSOgrant14', 'QSOgrant15', 'QSOgrant16',
       'QSOgrant17', 'QSOgrant18', 'QSOgrant19', 'QSOgrant20', 'PAAC7'],
      dtype='object', length=604)


Se importan las variables objetivo para aplicar los otros metódos de selección.

In [ ]:
pathpath = '/content/drive/MyDrive/Colab Notebooks/Laboratorio Integrado I/Variables objetivo/df_final.csv'
df_final_y = pd.read_csv(pathpath)

In [ ]:
df_final_y

,id,sequence,Antimicrobiano,Antifúngico,Antiviral,Anti Gram +,Anti Gram -,Anticancerigeno,Antibacteriano,AntiVIH,Antiparasitario,Antitumoral
0,peptide_0,GIGAVLKVLTTGLPALISWIKRKRQQ,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,peptide_1,GIGKFLHSAKKFGKAFVGEIMNS,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2,peptide_2,ILPWKWPWWPWRR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,peptide_3,KWCFRVCYRGICYRRCR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,peptide_4,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
40835,peptide_43335,SNQKQALAQPQTFAHIRVFFRPLPFAFFGTVGFYS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40836,peptide_43336,ETVDLKLNLQSNK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40837,peptide_43337,DRALAPNIRSAKQMVER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40838,peptide_43338,HDNKSKNNTFNGAIAYVIPFLLFIFVIRSLI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_final_y = df_final_y.iloc[:,2:]

In [ ]:
df_final_y

,Antimicrobiano,Antifúngico,Antiviral,Anti Gram +,Anti Gram -,Anticancerigeno,Antibacteriano,AntiVIH,Antiparasitario,Antitumoral
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
40835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##Aplicación del segundo método de selección

In [ ]:
y = df_final.iloc[:,2]

In [ ]:
y

,Antimicrobiano
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
40835,0.0
40836,0.0
40837,0.0
40838,0.0


In [ ]:
# Ajustar el clasificador Extra Trees
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(df_min_max, y)

model = SelectFromModel(clf, prefit=True)

# Transformar los datos para seleccionar características
X_new = model.transform(df_min_max)

# Obtener la máscara de características seleccionadas
selected_features_mask = model.get_support()

# Obtener los nombres de las características seleccionadas
selected_features = df_min_max.columns[selected_features_mask]

# Obtener los nombres de las características no seleccionadas (eliminadas)
eliminated_features = df_min_max.columns[~selected_features_mask]

# Mostrar los nombres de las columnas eliminadas
print(eliminated_features)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Index(['H', 'I', 'F', 'V', 'AA', 'AR', 'AN', 'AD', 'AC', 'AE',
       ...
       'QSOgrant39', 'QSOgrant40', 'QSOgrant41', 'QSOgrant44', 'QSOgrant48',
       'PAAC1', 'PAAC9', 'PAAC10', 'PAAC14', 'PAAC20'],
      dtype='object', length=1158)


In [ ]:
# Convertir X_new a un DataFrame y asignar los nombres de las columnas
X_new_df = pd.DataFrame(X_new, columns=selected_features)

print(f"Original shape: {X_reduced_df.shape}")
print(f"Reduced shape: {X_new_df.shape}")

Original shape: (40840, 903)
Reduced shape: (40840, 349)


In [ ]:
for i in selected_features:
  print(i)

Longitud
Indice_Boman
Carga
Punto_Isoelectrico
Hidrofobicidad
A
R
D
C
E
Q
G
H
I
L
K
M
F
P
T
W
Y
V
AK
CK
EE
GL
GK
IK
LG
LL
LK
KC
KI
KK
FL
SL
_PolarizabilityC1
_PolarizabilityC2
_PolarizabilityC3
_SolventAccessibilityC1
_SolventAccessibilityC2
_SolventAccessibilityC3
_SecondaryStrC1
_SecondaryStrC2
_SecondaryStrC3
_ChargeC1
_ChargeC2
_ChargeC3
_PolarityC1
_PolarityC2
_PolarityC3
_NormalizedVDWVC1
_NormalizedVDWVC2
_NormalizedVDWVC3
_HydrophobicityC1
_HydrophobicityC2
_HydrophobicityC3
_PolarizabilityT12
_PolarizabilityT23
_SolventAccessibilityT12
_SolventAccessibilityT13
_SolventAccessibilityT23
_SecondaryStrT12
_SecondaryStrT13
_SecondaryStrT23
_ChargeT12
_ChargeT23
_PolarityT12
_PolarityT13
_PolarityT23
_NormalizedVDWVT12
_NormalizedVDWVT13
_NormalizedVDWVT23
_HydrophobicityT12
_HydrophobicityT13
_HydrophobicityT23
_PolarizabilityD1001
_PolarizabilityD1025
_PolarizabilityD1050
_PolarizabilityD1075
_PolarizabilityD1100
_PolarizabilityD2001
_PolarizabilityD2025
_PolarizabilityD2050
_Pola

In [ ]:
X_new_df.head()

,Longitud,Indice_Boman,Carga,Punto_Isoelectrico,Hidrofobicidad,A,R,N,D,C,...,PAAC15,PAAC16,PAAC17,PAAC18,PAAC19,PAAC21,PAAC22,PAAC23,PAAC24,PAAC25
0,0.204301,0.283376,0.403184,0.877668,0.583464,0.106833,0.097162,0.000000,0.00000,0.000000,...,0.043584,0.037587,0.125689,0.062376,0.000000,0.252797,0.283954,0.276215,0.379835,0.403598
1,0.172043,0.274006,0.372405,0.723791,0.559056,0.120778,0.000000,0.083854,0.00000,0.000000,...,0.000000,0.079080,0.000000,0.000000,0.000000,0.387531,0.444310,0.371066,0.329478,0.328290
2,0.064516,0.314383,0.370923,0.875055,0.411451,0.000000,0.194336,0.000000,0.00000,0.000000,...,0.207112,0.000000,0.000000,0.494012,0.000000,0.524949,0.339267,0.526354,0.631409,0.368310
3,0.107527,0.467550,0.415301,0.658950,0.482135,0.000000,0.371518,0.000000,0.00000,0.431369,...,0.000000,0.000000,0.000000,0.074517,0.194805,0.554350,0.470860,0.593462,0.377683,0.423854
4,0.322581,0.434303,0.419405,0.771824,0.455650,0.000000,0.170702,0.052129,0.07567,0.000000,...,0.026058,0.044945,0.037566,0.000000,0.000000,0.423099,0.532257,0.415935,0.295583,0.472844


##Tercera y última técnica de selección de características.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Ver la forma original de X_new_df
print(f"Forma original de X_new_df: {X_new_df.shape}")

# Inicializar SelectKBest y ajustar a los datos
selector = SelectKBest(score_func=f_classif, k=245)
X_selected = selector.fit_transform(X_new_df, y)

# Obtener la máscara de características seleccionadas
selected_features_mask = selector.get_support()

# Obtener los nombres de las características seleccionadas
selected_columns = X_new_df.columns[selected_features_mask]

# Convertir X_selected a un DataFrame y asignar los nombres de las columnas
X_new_selected_df = pd.DataFrame(X_selected, columns=selected_columns)

# Imprimir la forma del nuevo DataFrame
print(f"Forma de X_new_selected_df: {X_new_selected_df.shape}")

# Identificar características eliminadas
removed_columns = X_new_df.columns[~selected_features_mask]

# Mostrar resultados
print("\nCaracterísticas seleccionadas:")
print(selected_columns.tolist())
print("\nCaracterísticas eliminadas:")
print(removed_columns.tolist())

Forma original de X_new_df: (40840, 349)
Forma de X_new_selected_df: (40840, 245)

Características seleccionadas:
['Longitud', 'Carga', 'Punto_Isoelectrico', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'K', 'M', 'S', 'T', 'W', 'Y', 'AK', 'CK', 'GL', 'GK', 'LG', 'LK', 'KC', 'KI', 'KL', 'KK', 'MT', 'TP', 'WK', '_PolarizabilityC1', '_PolarizabilityC3', '_SolventAccessibilityC1', '_SolventAccessibilityC3', '_SecondaryStrC3', '_ChargeC1', '_ChargeC2', '_ChargeC3', '_NormalizedVDWVC2', '_NormalizedVDWVC3', '_PolarizabilityT12', '_PolarizabilityT13', '_PolarizabilityT23', '_SolventAccessibilityT12', '_SolventAccessibilityT13', '_SolventAccessibilityT23', '_ChargeT12', '_ChargeT13', '_ChargeT23', '_PolarityT23', '_NormalizedVDWVT12', '_HydrophobicityT12', '_HydrophobicityT13', '_PolarizabilityD1001', '_PolarizabilityD1050', '_PolarizabilityD1075', '_PolarizabilityD1100', '_PolarizabilityD3050', '_PolarizabilityD3075', '_PolarizabilityD3100', '_SolventAccessibilityD1001', '_SolventAccessibilityD3001', '

In [ ]:
X_new_selected_df.shape

(40840, 245)

In [ ]:
X_new_selected_df.head(10)

,Longitud,Carga,Punto_Isoelectrico,R,N,D,C,E,Q,G,...,PAAC8,PAAC12,PAAC13,PAAC16,PAAC17,PAAC18,PAAC19,PAAC21,PAAC22,PAAC25
0,0.204301,0.403184,0.877668,0.097162,0.000000,0.000000,0.000000,0.000000,0.104891,0.144225,...,0.146147,0.141961,0.000000,0.037587,0.125689,0.062376,0.000000,0.252797,0.283954,0.403598
1,0.172043,0.372405,0.723791,0.000000,0.083854,0.000000,0.000000,0.090913,0.000000,0.217387,...,0.204963,0.199092,0.111045,0.079080,0.000000,0.000000,0.000000,0.387531,0.444310,0.328290
2,0.064516,0.370923,0.875055,0.194336,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.074941,0.000000,0.000000,0.000000,0.494012,0.000000,0.524949,0.339267,0.368310
3,0.107527,0.415301,0.658950,0.371518,0.000000,0.000000,0.431369,0.000000,0.000000,0.073525,...,0.058190,0.056523,0.000000,0.000000,0.000000,0.074517,0.194805,0.554350,0.470860,0.423854
4,0.322581,0.419405,0.771824,0.170702,0.052129,0.075670,0.000000,0.169531,0.036859,0.067563,...,0.058245,0.169730,0.000000,0.044945,0.037566,0.000000,0.000000,0.423099,0.532257,0.472844
5,0.161290,0.376791,0.875058,0.114833,0.000000,0.000000,0.000000,0.000000,0.000000,0.170450,...,0.159199,0.051534,0.000000,0.000000,0.136897,0.000000,0.000000,0.351507,0.470961,0.425579
6,0.064516,0.338658,0.701483,0.097162,0.000000,0.000000,0.000000,0.000000,0.000000,0.192312,...,0.186481,0.000000,0.000000,0.071949,0.000000,0.000000,0.000000,0.318908,0.387489,0.437260
7,0.182796,0.370916,0.723789,0.000000,0.000000,0.058338,0.000000,0.000000,0.000000,0.104163,...,0.104414,0.202865,0.000000,0.161157,0.000000,0.066855,0.000000,0.311487,0.378086,0.358945
8,0.193548,0.391431,0.729382,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,...,0.188469,0.183070,0.000000,0.036351,0.060767,0.060325,0.078852,0.358758,0.437773,0.419418
9,0.311828,0.382519,0.532108,0.140349,0.053576,0.038892,0.305564,0.000000,0.151514,0.034725,...,0.033007,0.032061,0.000000,0.076395,0.042577,0.042268,0.055249,0.372517,0.380198,0.256235


In [ ]:
X_new_selected_df.to_csv('X_selected', index=False)

**Fin de selección de características**

##Unir descriptores calculados con Python con los de R

In [ ]:
df_properties = df_properties.merge(df_descriptors, on='id', how='right')

In [ ]:
df_properties.head()

,id,sequence,Longitud,Indice_Boman,Carga,Punto_Isoelectrico,Hidrofobicidad,A,R,N,...,QSOgrant41,QSOgrant42,QSOgrant43,QSOgrant44,QSOgrant45,QSOgrant46,QSOgrant47,QSOgrant48,QSOgrant49,QSOgrant50
0,peptide_0,GIGAVLKVLTTGLPALISWIKRKRQQ,26,0.570000,4.997093,12.546070,0.273077,7.692,7.692,0.000,...,0.020989,0.014686,0.012446,0.006902,0.002686,0.000000,0.000000,0.000000,0.00000,0.000000
1,peptide_1,GIGKFLHSAKKFGKAFVGEIMNS,23,0.419130,3.089488,10.802733,0.082609,8.696,0.000,4.348,...,0.009988,0.001179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,peptide_2,ILPWKWPWWPWRR,13,1.069231,2.997683,12.516461,-1.069231,0.000,15.385,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,peptide_3,KWCFRVCYRGICYRRCR,17,3.535294,5.748073,10.068130,-0.517647,0.000,29.412,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,peptide_4,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,37,3.000000,6.002416,11.346921,-0.724324,0.000,13.514,2.703,...,0.025547,0.017740,0.022454,0.029931,0.022450,0.007209,0.014647,0.017532,0.01391,0.008937


In [ ]:
df_properties.to_csv('/content/drive/MyDrive/Colab Notebooks/AMP/df_descriptors.csv', index=False)

##Dataset final con características y variables objetivo

In [ ]:
pathpath = '/content/drive/MyDrive/Colab Notebooks/Laboratorio Integrado I/df_final.csv'
df_final = pd.read_csv(pathpath)

In [ ]:
df_final.iloc[:,2:12]

,Antimicrobiano,Antifúngico,Antiviral,Anti Gram +,Anti Gram -,Anticancerigeno,Antibacteriano,AntiVIH,Antiparasitario,Antitumoral
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
40835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_final

,id,sequence,Antimicrobiano,Antifúngico,Antiviral,Anti Gram +,Anti Gram -,Anticancerigeno,Antibacteriano,AntiVIH,Antiparasitario,Antitumoral,Sufficient_AA_Diversity
0,peptide_0,GIGAVLKVLTTGLPALISWIKRKRQQ,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
1,peptide_1,GIGKFLHSAKKFGKAFVGEIMNS,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,True
2,peptide_2,ILPWKWPWWPWRR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
3,peptide_3,KWCFRVCYRGICYRRCR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,True
4,peptide_4,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40835,peptide_43335,SNQKQALAQPQTFAHIRVFFRPLPFAFFGTVGFYS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
40836,peptide_43336,ETVDLKLNLQSNK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
40837,peptide_43337,DRALAPNIRSAKQMVER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
40838,peptide_43338,HDNKSKNNTFNGAIAYVIPFLLFIFVIRSLI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [ ]:
path_df_Y = '/content/drive/MyDrive/Colab Notebooks/AMP/df_final_positive.csv'

In [ ]:
df_final_positive = pd.read_csv(path_df_Y)

In [ ]:
df_final_positive.shape

(23105, 12)

In [ ]:
df_complete = df_properties.merge(df_final_positive, on=['id', 'sequence'], how='right')

In [ ]:
df_complete.to_csv('df_complete.csv', index=False)